In [ ]:
pip install beautifulsoup4

In [48]:
# 필요한 모듈과 라이브러리
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from datetime import date

# 오늘 날짜
today = date.today()

# 메인 페이지 크롤러
def geumcheon_page_crawler(soup):
    for data in soup.select('#contents > div > table > tbody > tr'):
        # 카테고리 (분야)
        cat_temp = data.select('td')[1].text.strip() if data.select('td') else 'N/A'
        # "임신/출산" 또는 "영유아" 카테고리에 해당하는 경우에만 정보를 수집
        if cat_temp in ['임신/출산', '영유아']:
            category_list.append(cat_temp)

            # 지원사업명 (제목)
            title_temp = data.select('td.p-subject > a')[0].text.strip() if data.select('td.p-subject > a') else 'N/A'
            title_list.append(title_temp)

            # 크롤링할 세부 URL
            if data.select('td.p-subject > a'):
                relative_url = data.select('td.p-subject > a')[0]['href']
                url_temp = 'https://www.geumcheon.go.kr/portal/' + relative_url
            else:
                url_temp = 'N/A'
            url_list.append(url_temp)

            # 주최 (국가/구)
            host_temp = data.select('td')[3].text.strip() if data.select('td') else 'N/A'
            host_list.append(host_temp)

# 세부 페이지 크롤러
def geumcheon_target_page_crawler():
    for i in range(len(url_list)):
        if url_list[i] != 'N/A':  # 유효한 URL만 요청
            target_response = requests.get(url_list[i])
            target_html = target_response.text
            soup = BeautifulSoup(target_html, 'html.parser')

            # 지원사업명 (제목)
            title_temp = soup.select_one('#contents > div > table > tbody > tr:nth-child(4) > td')
            title_text = title_temp.get_text(strip=True) if title_temp else 'N/A'
            title_list[i] = title_text  # 기존 제목을 세부 페이지에서 크롤링한 제목으로 덮어쓰기

            # 지원대상
            field_temp = soup.select('#contents > div > table > tbody > tr:nth-child(5) > td')[0].text.strip() if soup.select('#contents > div > table > tbody > tr:nth-child(5) > td') else 'N/A'
            field_list.append(field_temp)

            # 내용
            description_temp = soup.select('#contents > div > table > tbody > tr:nth-child(6) > td')[0].text.strip() if soup.select('#contents > div > table > tbody > tr:nth-child(6) > td') else 'N/A'
            description_list.append(description_temp)

            # 신청방법
            way_temp = soup.select('#contents > div > table > tbody > tr:nth-child(7) > td')[0].text.strip() if soup.select('#contents > div > table > tbody > tr:nth-child(7) > td') else 'N/A'
            way_list.append(way_temp)

            # 문의처
            inquiry_temp = soup.select('#contents > div > table > tbody > tr:nth-child(8) > td')[0].text.strip() if soup.select('#contents > div > table > tbody > tr:nth-child(8) > td') else 'N/A'
            inquiry_list.append(inquiry_temp)

            # 기타
            etc_temp = soup.select('#contents > div > table > tbody > tr:nth-child(10) > td')[0].text.strip() if soup.select('#contents > div > table > tbody > tr:nth-child(10) > td') else 'N/A'
            etc_list.append(etc_temp)

            # 관련 홈페이지
            homepage_temp = soup.select('#contents > div > table > tbody > tr:nth-child(9) > td > a')[0]['href'] if soup.select('#contents > div > table > tbody > tr:nth-child(9) > td > a') else 'N/A'
            homepage_list.append(homepage_temp)

# 금천구 맞춤복지 검색 사이트
url = 'https://www.geumcheon.go.kr/portal/selectBbsNttList.do?key=3892&id=&bbsNo=150544'
page_num = 1
last_page_reached = False

# 정보 담을 리스트 초기화
category_list = []      # 카테고리 (분야)
host_list = []          # 주최 (국가/구)
title_list = []         # 지원사업명 (제목)
field_list = []         # 지원대상
description_list = []   # 내용
way_list = []           # 신청방법
inquiry_list = []       # 문의처
etc_list = []           # 기타
url_list = []           # 크롤링한 세부 url
homepage_list = []      # 관련 홈페이지

# 크롤링 걸린 시간
tik = time.time()

# 페이지가 존재할 때까지 반복
while not last_page_reached:
    page_url = url + '&pageIndex=' + str(page_num)
    response = requests.get(page_url)

    # 페이지 존재 확인 (없으면 break)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.text, 'html.parser')

    # 더이상 추가 데이터 없는지 확인
    if not soup.select('#contents > div > table > tbody > tr'):
        break

    # 정보 수집
    geumcheon_page_crawler(soup)
    page_num += 1

# 크롤링 종료
print("---{} seconds done---".format(time.time() - tik))

# 메인 페이지 종료 이후, 세부 페이지 크롤링
geumcheon_target_page_crawler()

# 데이터 프레임 생성
df = pd.DataFrame({'분야': category_list,
                   '주최': host_list,
                   '제목': title_list,
                   '대상': field_list,
                   '내용': description_list,
                   '신청방법': way_list,
                   '문의처': inquiry_list,
                   '기타': etc_list,
                   '원본 url': url_list,
                   '관련 홈페이지': homepage_list})

df.index = df.index + 1

# 저장할 디렉토리 설정 / 없으면 생성
save_dir = '../[금천구]/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# CSV 파일 저장
df.to_csv(save_dir + "금천구_임신_출산_영유아_" + str(today) + '_검색결과.csv', encoding='utf-8-sig')

# 데이터 프레임 저장 완료
print('저장 완료')


---32.363407135009766 seconds done---
저장 완료
